## Finding the most profitable towns in a CCAA given a max price

In [2]:
import pandas as pd
import math
from bs4 import BeautifulSoup
import requests
import nums_from_string as nfs
import numpy as np
from re import search
from random import randint
from time import sleep
import openpyxl
import xlrd
import lxml

#### Importing geo data from csv file

In [3]:
geo_data = pd.read_csv('/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/data/province-town-n_props/geo-data_2021.10.23.csv')

In [4]:
geo_data.head(3)

,Unnamed: 0,town,n_properties,province,ccaa
0,0,alegria___dulantzi,26,alava,pais_vasco
1,1,amurrio,6,alava,pais_vasco
2,2,anana,2,alava,pais_vasco


#### Get input from user

In [11]:
ccaa_lst = geo_data['ccaa'].unique().tolist()
ccaa = input("ccaa: ")
while ccaa not in ccaa_lst:
    ccaa = input("There was no match between your input and our ccaa, try again: ")

ccaa: pais vasco
There was no match between your input and our ccaa, try again: pais_vasco


In [17]:
max_price = ""
while max_price == "":
    try:
        max_price = int(input("maximum price: "))
    except: 
        max_price = ""

maximum price: a
maximum price: a
maximum price: a
maximum price: a
maximum price: a
maximum price: a
maximum price: a
maximum price: a
maximum price: a
maximum price: 10


#### List of towns that we need to extract data from

In [22]:
towns = geo_data[geo_data['ccaa'] == ccaa].town.to_list()

In [23]:
towns

['alegria___dulantzi',
 'amurrio',
 'anana',
 'aramaio',
 'arminon',
 'arraia___maeztu',
 'arrazua___ubarrundia',
 'artziniega',
 'asparrena',
 'ayala___aiara',
 'barrundia',
 'berantevilla',
 'bernedo',
 'campezo___kanpezu',
 'elburgo___burgelu',
 'elciego',
 'elvillar___bilar',
 'harana___valle_de_arana',
 'iruna_oka___iruna_de_oca',
 'iruraiz___gauna',
 'kripan',
 'kuartango',
 'labastida',
 'lagran',
 'laguardia',
 'lanciego___lantziego',
 'lantaron',
 'lapuebla_de_labarca',
 'legutiano',
 'leza',
 'llodio',
 'moreda_de_alava',
 'okondo',
 'oyon___oion',
 'penacerrada___urizaharra',
 'ribera_alta',
 'ribera_baja___erribera_beitia',
 'salvatierra_o_agurain',
 'san_millan___donemiliaga',
 'urkabustaiz',
 'valdegovia',
 'vitoria___gasteiz',
 'zalduondo',
 'zambrana',
 'zigoitia',
 'zuia',
 'abaltzisketa',
 'aduna',
 'aia',
 'aizarnazabal',
 'albiztur',
 'alegia',
 'alkiza',
 'altzaga',
 'altzo',
 'amezketa',
 'andoain',
 'anoeta',
 'antzuola',
 'arama',
 'aretxabaleta',
 'arrasate_o_m

Hay que hacer un for loop que para cada town saque el df y le haga un append a el global.

#### Extracting data from each town

In [5]:
inmueble = "viviendas"
municipio = "hospitalet_de_llobregat"
ascensor = "-ascensor"

habitaciones = "hab="+"1"
baños = "&"+"ban="+"1"
precio_maximo = "&"+"pmax="+"90000"
metros = "m2="+"50"

In [7]:
url = "https://www.habitaclia.com/"+inmueble+ascensor+"-"+municipio+".htm?"+habitaciones+baños+precio_maximo

In [8]:
print(url)

https://www.habitaclia.com/viviendas-ascensor-hospitalet_de_llobregat.htm?hab=1&ban=1&pmax=90000


In [9]:
r = requests.get(url)
r.status_code

200

In [10]:
soup = BeautifulSoup(r.content, 'html.parser')

In [11]:
total_results = int(soup.find('h2', attrs={'class': 'f-right'}).find('span').get_text().replace(".",""))
total_results

26

In [12]:
pages = range(int(round(total_results/16)))

In [13]:
pages = range(2)

In [14]:
inmuebles = []
for p in pages:
    sleep(randint(3,7))
    url = "https://www.habitaclia.com/"+inmueble+ascensor+"-"+municipio+"-"+str(p)+".htm?"+habitaciones+"&"+baños+"&"+precio_maximo
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        inmuebles += soup.find_all('div', attrs={'class': 'list-item-info'})
        del inmuebles[-1] # Last item is an ad
    except:
        print(f'Error on page {start}.')
        continue
    print("page ", p)
len(inmuebles)

page  0
page  1


47

In [15]:
name = []
town = []
area = []
features = []
m2 = []
n_rooms = []
n_bath = []
price_m2 = []
description = []
price = []
opportunity = []
price_reduction = []
opportunity = []
last_update = []
url = []

for i,inmuebles in enumerate(inmuebles):
    #print("inmueble ", i)
    
    ## fetching info
    try:
        name_temp = inmuebles.find('h3', attrs={'class': 'list-item-title'}).get_text(strip=True)
        town_temp = inmuebles.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).split("-",1)[0].strip()
        area_temp = inmuebles.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).split("-",1)[1].strip().replace("Ver mapa","")
        
        m2_temp = nfs.get_nums(inmuebles.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[0])[0] 
        n_rooms_temp = nfs.get_nums(inmuebles.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[1])[0]
        n_bath_temp = nfs.get_nums(inmuebles.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[2])[0] 
        price_m2_temp = nfs.get_nums(inmuebles.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[3])[0] 

        raw_price_temp = inmuebles.find('article', attrs={'class': 'list-item-price'}).get_text()
        if search("Oportunidad", raw_price_temp):
            if search("ha bajado", raw_price_temp): 
                #print("1.1")
                price_temp = nfs.get_nums((inmuebles.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                price_reduction_temp = nfs.get_nums((inmuebles.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]
                opportunity_temp = "yes"
            else:
                #print("1.2")
                price_temp = nfs.get_nums((inmuebles.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                price_reduction_temp = "0"
                opportunity_temp = "yes"
            
        elif search("ha bajado", raw_price_temp):
            #print("2")
            price_temp = nfs.get_nums((inmuebles.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
            price_reduction_temp = nfs.get_nums((inmuebles.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]      
            opportunity_temp = "no"
            
        else: 
            #print("3")
            price_temp = nfs.get_nums((inmuebles.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
            price_reduction_temp = "0"
            opportunity_temp = "no"

        description_temp = inmuebles.find('p', attrs={'class': 'list-item-description'}).get_text(strip=True)
        last_update_temp = nfs.get_nums(inmuebles.find('span', attrs={'class': 'list-item-date'}).get_text(strip=True))
        url_temp = inmuebles.find('h3', attrs={'class': 'list-item-title'}).find('a').get('href')

        ## appending info to each list
        name.append(name_temp)
        town.append(town_temp)
        area.append(area_temp)
        #features.append(features_temp)
        m2.append(m2_temp)
        n_rooms.append(n_rooms_temp)
        n_bath.append(n_bath_temp)
        price_m2.append(price_m2_temp)
        price.append(price_temp)
        opportunity.append(opportunity_temp)
        price_reduction.append(price_reduction_temp)
        description.append(description_temp)
        last_update.append(last_update_temp)
        url.append(url_temp)
        
        #print("\n")
    
    except:
        print(name_temp,"\n", i,"\n", "error", "\n")
        """
        print("name_temp","\n", name_temp,"\n")
        print("town_temp","\n", town_temp,"\n")
        print("area_temp","\n",area_temp,"\n")
        #print("features_temp","\n",features_temp,"\n")
        print("m2_temp","\n",m2_temp,"\n")
        print("n_rooms_temp","\n",n_rooms_temp,"\n")
        print("n_bath_temp","\n",n_bath_temp,"\n")
        print("price_m2_temp","\n",price_m2_temp,"\n")
        print("price_temp","\n",price_temp,"\n")
        print("opportunity_temp","\n",opportunity_temp,"\n")
        print("price_reduction_temp","\n",price_reduction_temp,"\n")
        print("description_temp","\n",description_temp,"\n")
        print("last_update_temp","\n",last_update_temp,"\n")
        print("url_temp","\n",url_temp,"\n")
        """

In [16]:
x = min(len(name), len(town), len(area), len(description), len(price), len(last_update), len(url))
dct = {'name': name[:x], 'town': town[:x], 'area': area[:x], 'm2': m2[:x], 'n_rooms': n_rooms[:x], 'n_bath': n_bath[:x], 'price_m2': price_m2[:x] ,'price': price[:x], 'price_reduction': price_reduction[:x], 'opportunity':opportunity[:x], 'last_update': last_update[:x],  'description': description[:x], 'url':url[:x] }

df = pd.DataFrame.from_dict(dct)
df

,name,town,area,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url
0,Piso en Les Planes. Piso 50 mâ² en hospitale...,Hospitalet de Llobregat (L´),Les Planes,50,2,1,1.780,89000,10000,no,[],"Piso 50 mÂ² en Hospitalet de Llobregat (LÂ´), ...",https://www.habitaclia.com/comprar-piso-50_ma2...
1,Piso en Pubilla Cases. Oportunidad!,Hospitalet de Llobregat (L´),Pubilla Cases,50,2,1,1.798,89900,5100,no,[3],OPORTUNIDAD! Vivienda de 50m2. Dispone de sal...,https://www.habitaclia.com/comprar-piso-oportu...
2,Piso en Santa Eulàlia. Dos habitaciones y as...,Hospitalet de Llobregat (L´),Santa Eulàlia,45,2,1,2.200,99000,0,no,[17],DOS HABITACIONES Y ASCENSOR Decimoprimera pla...,https://www.habitaclia.com/comprar-piso-dos_ha...
3,Piso Blocs de la florida. Piso en plaza blocs...,Hospitalet de Llobregat (L´),Les Planes,57,3,1,1.140,65000,4000,no,[11],Piso de tres dormitorios y un cuarto de baño. ...,https://www.habitaclia.com/comprar-piso-en_pla...
4,Piso Carrer d'amadeu vives. Bajos exterior re...,Hospitalet de Llobregat (L´),Pubilla Cases,50,1,1,1.460,73000,3000,no,[8],"Piso en planta baja con acesso directo, totalm...",https://www.habitaclia.com/comprar-piso-bajos_...
5,Ático en Santa Eulàlia. Ático con 2 habitaci...,Hospitalet de Llobregat (L´),Santa Eulàlia,74,2,1,608.000,45000,0,no,[],OPORTUNIDAD INVERSORES -VENTA NUDA PROPIEDAD. ...,https://www.habitaclia.com/comprar-atico-con_2...
6,Piso Lavinia. Piso en calle lavinia,Hospitalet de Llobregat (L´),Santa Eulàlia,42,2,1,2.262,95000,0,no,[11],Piso de dos dormitorios y un cuarto de baño. S...,https://www.habitaclia.com/comprar-piso-en_cal...
7,Piso Carrer de lavínia. Vivienda para reformar,Hospitalet de Llobregat (L´),Santa Eulàlia,42,2,1,2.262,95000,0,no,[3],Piso en venta para REFORMAR .Se encuentra en l...,https://www.habitaclia.com/comprar-piso-vivien...
8,Planta baja Carrer cinca. Increible ubicacion,Hospitalet de Llobregat (L´),Collblanc,35,1,1,1.571,55000,0,no,[],OPORTUNIDAD PARA INVERTIR!! La Casa Agency ve...,https://www.habitaclia.com/comprar-planta_baja...
9,Piso Carrer piera. Precioso piso de 3 habitac...,Hospitalet de Llobregat (L´),La Florida,60,3,1,1.433,86000,3700,no,[13],Piso muy luminoso de 3 habitaciones 1 doble. P...,https://www.habitaclia.com/comprar-piso-precio...


#### Appending df of the town to the CCAA df

#### Filtering those towns that don't meet the treshold of min properties to be statistically relevant